In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [5]:
# Define the model
model = keras.Sequential([
    # Convolutional Layers
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(640, 640, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Flatten the feature maps
    layers.Flatten(),

    # Dense Layers
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Output layer with sigmoid activation for binary classification
])

In [6]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Use binary cross-entropy for binary classification
              metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 638, 638, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 319, 319, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 317, 317, 64)      18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 158, 158, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 156, 156, 128)     73856     
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 78, 78, 128)      

In [4]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="BXRbaNxc30Qj54DB0D0O")
project = rf.workspace("dataset-5cq1l").project("person-amhwo")
dataset = project.version(1).download("folder")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.1
    Uninstalling pyparsing-3.1.1:
      Successfully uninstalled pyparsing-3.1.1
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.8.1.78
    Uninstalling opencv-python-headless-4.8.1.78:
      Successfully uninstalled 

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to person-1 in folder:: 100%|██████████| 90/90 [00:00<00:00, 4972.24it/s]


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the data generators for training and validation
train_data_dir = '/content/person-1/train'
validation_data_dir = '/content/person-1/valid'

# Define data augmentation and preprocessing for training images
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to [0, 1]
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# No data augmentation for validation images, only rescaling
validation_datagen = ImageDataGenerator(rescale=1./255)

# Batch size and image dimensions
batch_size = 2
img_height = 640
img_width = 640

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'  # Use 'binary' for binary classification
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'  # Use 'binary' for binary classification
)

# Train the model
num_epochs = 50

history = model.fit(
    train_generator,
    epochs=num_epochs,
    validation_data=validation_generator
)

# Save the trained model
model.save('person_classification_model.h5')


Found 81 images belonging to 1 classes.
Found 3 images belonging to 1 classes.
Epoch 1/50
41/41 [==============================] - 25s 235ms/step - loss: 0.1306 - accuracy: 0.9506 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/50
41/41 [==============================] - 12s 303ms/step - loss: 6.9128e-19 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/50
41/41 [==============================] - 10s 252ms/step - loss: 1.8098e-18 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/50
41/41 [==============================] - 9s 213ms/step - loss: 7.3158e-23 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/50
41/41 [==============================] - 10s 252ms/step - loss: 3.5271e-33 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/50
41/41 [==============================] - 10s 247ms/step - loss: 6.5210e-24 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/50
41/41 [==

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
